In [ ]:
import os
import pandas as pd
from pathlib import Path
import google.generativeai as genai
import mimetypes
import json
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe
from datetime import datetime
import pytz
from googleapiclient.http import MediaIoBaseDownload
from io import BytesIO
import requests
from pathlib import Path
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

In [ ]:
genai.configure(api_key='Pass grmini key') #

MODEL_CONFIG = {
    "temperature": 0,
    "top_p": 1,
    "top_k": 32,
    "max_output_tokens": 8000,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    generation_config=MODEL_CONFIG,
    safety_settings=safety_settings,
)


# Service Account Credentials
service_account_info = { # pass service account creds
}

creds = Credentials.from_service_account_info(service_account_info, scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)
gc = gspread.authorize(creds)

SHEET_ID = '1no0oP-U8dMPv6LuR49pEcDLg-cyUl9XcMiyG-5gHYAI'
sheet = gc.open_by_key(SHEET_ID).sheet1

SPREADSHEET_ID = '1no0oP-U8dMPv6LuR49pEcDLg-cyUl9XcMiyG-5gHYAI'
sheet2 = gc.open_by_key(SPREADSHEET_ID).worksheet('Links')
data = sheet2.col_values(1)


In [ ]:
links = list(set([line for line in data if line.startswith("https://public-assets-dev.meredithcorp.io/")]))
clean_links = list(set([link.split(" (edited)")[0] for link in links]))

print("Cleaned Links:", clean_links)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


month = "Mar"
DOWNLOAD_DIR = f'/content/drive/My Drive/Images/Downloads/{month}'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

total_links = len(clean_links)
successful_downloads = 0
failed_downloads = 0

for link in clean_links:
    try:
        response = requests.get(link, stream=True)
        file_name = link.split('/')[-1]
        file_path = os.path.join(DOWNLOAD_DIR, file_name)

        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded and saved to Google Drive: {file_name}")
            successful_downloads += 1
        else:
            print(f"Failed to download: {link} (Status Code: {response.status_code})")
            failed_downloads += 1
    except Exception as e:
        print(f"Error processing {link}: {e}")
        failed_downloads += 1

print(f"Total Links: {total_links}")
print(f"Successful Downloads: {successful_downloads}")
print(f"Failed Downloads: {failed_downloads}")

Cleaned Links: ['https://public-assets-dev.meredithcorp.io/ffa8dd9decaf6bbf8f416c6bf9254440/03-28-2025-08-56-35-screenshot.png', 'https://public-assets-dev.meredithcorp.io/ff610e9b83e27d779cf6f2b78bb35695/03-27-2025-03-54-45-screenshot.png']
Mounted at /content/drive
Downloaded and saved to Google Drive: 03-28-2025-08-56-35-screenshot.png
Downloaded and saved to Google Drive: 03-27-2025-03-54-45-screenshot.png
Total Links: 2
Successful Downloads: 2
Failed Downloads: 0


In [ ]:
def get_subfolder_id(parent_folder_id, subfolder_name):
  query = f"'{parent_folder_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and name = '{subfolder_name}'"
  results = service.files().list(q=query, spaces='drive').execute()
  folders = results.get('files', [])
  if not folders:
      raise FileNotFoundError(f"Subfolder named '{subfolder_name}' not found in folder {parent_folder_id}")
  return folders[0]['id']

parent_folder_id = '1HLUafZ-MJToA4B5162uA67IZPlDnvhTO'
subfolder_name = month
subfolder_archive_name = f'{month}_archive'

try:
    subfolder_id = get_subfolder_id(parent_folder_id, subfolder_name)
    print(f"Subfolder ID for '{subfolder_name}': {subfolder_id}")
except FileNotFoundError as e:
    print(e)
    subfolder_id = None

try:
    archive_folder_id = get_subfolder_id(parent_folder_id, subfolder_archive_name)
    print(f"Subfolder ID for archive '{subfolder_archive_name}': {archive_folder_id}")
except FileNotFoundError as e:
    print(e)
    archive_folder_id = None

Subfolder ID for 'Mar': 1C_aqK9JrE2uIX3ABb-BGI13iT9t5OdOp
Subfolder ID for archive 'Mar_archive': 1-67kzfUnefW8fDxLPbfMdNANWuswhq0E


In [ ]:
folder_id = subfolder_id #'1Z2vrAmOEFSAzHSsLrMaBxxanFtVOTtfD' #subfolder_id

archive_folder_id = archive_folder_id #'1-Ers6mh3KNjg-zgdplmUiOMYXxDko9H9' #archive_folder_id

In [ ]:
def list_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents"
    results = service.files().list(q=query, spaces='drive').execute()
    return results.get('files', [])

# Function to move file to Archive folder
def move_file_to_archive(file_id, archive_folder_id):
    file = service.files().get(fileId=file_id, fields="parents").execute()
    previous_parents = ",".join(file.get('parents'))

    updated_file = service.files().update(fileId=file_id,
                                          addParents=archive_folder_id,
                                          removeParents=previous_parents,
                                          fields='id, parents').execute()
    print(f"Moved file {file_id} to archive folder.")
    return updated_file


def gemini_output(image_path, system_prompt, user_prompt):
    image_info = image_format(image_path)
    input_prompt = [system_prompt, image_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text

def image_format(image_path):
    img = Path(image_path)

    if not img.exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/jpeg",
            "data": img.read_bytes()
        }
    ]
    return image_parts

def download_image_from_drive(file_id, destination_path):
    request = service.files().get_media(fileId=file_id)
    fh = BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()
    fh.seek(0)
    with open(destination_path, 'wb') as f:
        f.write(fh.read())
    print(f"Downloaded {destination_path} from Google Drive.")
    return destination_path


def process_image_folder(folder_id, archive_folder_id, system_prompt, user_prompt):
    json_outputs = []
    files = list_files_in_folder(folder_id)
    total_images = len([f for f in files if f['name'].endswith((".jpg", ".jpeg", ".png"))])
    processed_count = 0

    for idx, file in enumerate(files, start=1):
        if file['name'].endswith((".jpg", ".jpeg", ".png")):
            image_id = file['id']
            image_name = file['name']
            try:
                print(f"Processing ({idx}/{total_images}): {image_name}")

                local_image_path = f'/content/{image_name}'

                download_image_from_drive(image_id, local_image_path)

                output = gemini_output(local_image_path, system_prompt, user_prompt)
                print(f"JSON Output for {image_name}:")
                print(output)
                json_outputs.append({"image_name": image_name, "output": output})

                move_file_to_archive(image_id, archive_folder_id)

                processed_count += 1
            except Exception as e:
                print(f"Error processing {image_name}: {e}")

    print(f"Processed {processed_count} out of {total_images} images.")
    return json_outputs

In [ ]:
system_prompt = """Convert Invoice data into json format with appropriate json tags as required for the data in image give only in this format for all articles {
    "Screenshot_taken_date": "12-19-2024",
    "Screenshot_taken_time": "11:26 AM",
    "Rank": "1",
    "Title": "InStyle",
    "article": "Lady Gaga's massive engagement ring is now estimated to be worth over $1 million",
    "is AN+?": if News+ mark available "Yes" else "No",
    "Category": "Entertainment"
  }"""
user_prompt = "Convert image data into json format with appropriate json tags only take Trending story section from the image which will have only 6 article with ranking and only identify Screenshot taken time - screenshot taken time shud only be in 12hr format(HH:MM AM/PM), Screenshot taken date, Rank, Title, article, wheather it is Apple news+ if yes mention yes else NO and give the Screenshot_taken_date format like this MM-DD-YYYY Year is 2025 for all articles within the trending stories and add one more column as Category with which the article falls under any of this (Health,Tech,Lifestyle,Entertainment & Celebrity,Science & Nature,Business,Sports,Home,Middle East,Finance,World,Travel,Politics,Military,Crime,Shopping,Real Estate,Royals,Food & Cooking,United States,Jobs,Other,Fashion & Beauty). Please be accurate with all the columns values dont mingle it"

json_outputs = process_image_folder(folder_id, archive_folder_id, system_prompt, user_prompt)


cleaned_json_outputs = []
for item in json_outputs:
    cleaned_item = item["output"].strip('```json\n').strip('```')
    try:
        parsed_json = json.loads(cleaned_item)
        if isinstance(parsed_json, list):
            for entry in parsed_json:
                entry["image_name"] = item["image_name"]
            cleaned_json_outputs.extend(parsed_json)
        elif isinstance(parsed_json, dict):
            parsed_json["image_name"] = item["image_name"]
            cleaned_json_outputs.append(parsed_json)
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON for {item['image_name']}: {e}")

rows = []
for entry in cleaned_json_outputs:
    row = {
        'Image_Name': entry.get('image_name', ''),
        'Screenshot_taken_time': entry.get('Screenshot_taken_time', ''),
        'Screenshot_taken_date': entry.get('Screenshot_taken_date', ''),
        'Rank': entry.get('Rank', ''),
        'Title': entry.get('Title', ''),
        'Article': entry.get('article', ''),
        'is AN+?': entry.get('is AN+?', entry.get('is AN+', '')),
        'Category': entry.get('Category', ''),
    }
    rows.append(row)

df = pd.DataFrame(rows)
df = df[df.groupby('Image_Name')['Image_Name'].transform(lambda x: len(x)) >= 6]
ist = pytz.timezone('Asia/Kolkata')
current_time_ist = datetime.now(ist).strftime('%Y-%m-%d %H:%M:%S')
df['Date_Added'] = current_time_ist
category_max = df.groupby("Article", as_index=False)["Category"].max()
df = pd.merge(df.drop("Category", axis=1), category_max, on="Article", how="left")
df["Title"] = df["Title"].str.title()
columns_order = ['Image_Name', 'Screenshot_taken_time', 'Screenshot_taken_date', 'Rank', 'Title', 'Article', 'is AN+?', 'Category', 'Date_Added']
df = df[columns_order]

last_row = len(sheet.get_all_values())
set_with_dataframe(sheet, df, row=last_row+1, col=1, include_column_header=False)


print("New data has been appended to the Google Sheets.")

Processing (1/13): 03-03-2025-17-26-23-screenshot.png
Downloaded /content/03-03-2025-17-26-23-screenshot.png from Google Drive.
JSON Output for 03-03-2025-17-26-23-screenshot.png:
```json
[
  {
    "Screenshot_taken_date": "03-03-2025",
    "Screenshot_taken_time": "05:26 PM",
    "Rank": "1",
    "Title": "THE NEW REPUBLIC",
    "article": "JD Vance Blasted by Protesters Over Zelenskiy Blow Up",
    "is AN+?": "No",
    "Category": "Politics"
  },
  {
    "Screenshot_taken_date": "03-03-2025",
    "Screenshot_taken_time": "05:26 PM",
    "Rank": "2",
    "Title": "ABC NEWS",
    "article": "Selma Blair's service dog is Oscars party's surprise star",
    "is AN+?": "No",
    "Category": "Entertainment"
  },
  {
    "Screenshot_taken_date": "03-03-2025",
    "Screenshot_taken_time": "05:26 PM",
    "Rank": "3",
    "Title": "VARIETY",
    "article": "John Oliver Sounds Off on Trump Calling Zelenskyy Not Grateful Enough for U.S. Support: 'He's a President in the Middle...",
    "is AN+?"